In [1]:
import os
import numpy as np

ROOT = "/home/mario/Python/Datasets/EEG/timeseries/processed/rqa_windows/"
RAW = os.path.join(ROOT, "raw")
PROCESSED = os.path.join(ROOT, "dataset_indexed")
ELECTRODE_INDEXED = os.path.join(ROOT, "electrode_indexed")
if not os.path.exists(PROCESSED):
    os.makedirs(PROCESSED)


data = {f.split("_")[2]: os.path.join(ROOT, RAW, f) for f in os.listdir(RAW)}
folders = ['DD_DOWN', 'CT_UP', 'DD_UP', 'CT_DOWN']
for key, value in data.items():
    data[key] = [os.path.join(value, folders[i], f"rqa_analysis_{folders[i]}_metrics.npz") for i in range(4)]
# Explore one of the files to understand the window sizes used
window_sizes = np.load(data[list(data.keys())[0]][0], allow_pickle=True)["window_sizes"]



In [ ]:
def reorganize_data(data, window_sizes, electrode_indexed_dir):
    """
    Reorganize EEG data by window size
    
    Parameters:
    -----------
    data : dict
        Dictionary with electrode channels as keys and lists of file paths as values
    window_sizes : list
        List of window sizes
    electrode_indexed_dir : str
        Root directory for electrode-indexed output files
    """
    # Process each window size
    for w in window_sizes:
        print(f"Processing window size {w}...")
        # Create window size directory if it doesn't exist
        window_dir = os.path.join(electrode_indexed_dir, f"window_{w}")
        up_dir = os.path.join(window_dir, "up")
        down_dir = os.path.join(window_dir, "down")
        os.makedirs(up_dir, exist_ok=True)
        os.makedirs(down_dir, exist_ok=True)
        os.makedirs(window_dir, exist_ok=True)
        
        # Keys for this window size
        metric_key = f"w{w}_metrics"
        centers_key = f"w{w}_centers"
        stride_key = f"w{w}_stride"
        
        # Get common data (from first electrode, first condition file)
        first_electrode = list(data.keys())[0]
        first_file = data[first_electrode][0]
        common_data = np.load(first_file, allow_pickle=True)
        
        # Save metadata (common fields)
        metadata = {
            "metric_names": common_data["metric_names"],
            "centers": common_data[centers_key],
            "stride": common_data[stride_key]
        }
        np.savez(os.path.join(window_dir, "metadata.npz"), **metadata)
        
        # Process each electrode and condition
        for electrode, files in data.items():
            for file_path in files:
                # Get condition from file path
                condition = file_path.split("/")[-2]  # DD_DOWN, CT_UP, etc.
                
                # Load data
                npz_data = np.load(file_path, allow_pickle=True)
                
                # Extract only metrics for this window size
                metrics = {
                    "metrics": npz_data[metric_key]
                }
                
                # Save to new file
                if "up" in condition:
                    window_dir = up_dir
                else:
                    window_dir = down_dir
                output_file = os.path.join(window_dir, f"{electrode}_{condition}.npz")
                np.savez(output_file, **metrics)
        
    print(f"Data reorganization complete. Files saved to {electrode_indexed_dir}")

reorganize_data(data=data, window_sizes=window_sizes, electrode_indexed_dir=ELECTRODE_INDEXED)

Processing window size 50...
Processing window size 100...
Processing window size 150...
Processing window size 200...
Data reorganization complete. Files saved to /home/mario/Python/Datasets/EEG/timeseries/processed/rqa_windows/electrode_indexed
